In [1]:
import numpy as np
import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
class GFuzzyMinMaxNN:
    
    def __init__(self,sensitivity,theta=0.4):
        self.gamma = sensitivity
        self.hyperboxes = {}
        self.clasess = None
        self.V = []
        self.W = []
        self.U = []
        self.hyperbox_class = []
        self.theta = theta
        
        
    def func(self,r,gamma):
        if r*gamma>1:
            return 1
        elif 0<=r*gamma<=1 :
            return r*gamma
        else:
            return 0

    def fuzzy_membership(self,x,v,w):
        gamma = self.gamma
        xh_l , xh_u = x[0],x[1]
        return min([min(1-self.func(xh_u[i]-w[i] , gamma ) , 1-self.func(v[i]-xh_l[i] , gamma) ) for i in range(len(v))])

    
    def get_hyperbox(self,x,d):
        xh_l,xh_u = x[0],x[1]
        tmp = [0 for i in range(self.clasess)]
        tmp[d[0]-1] = 1
        
        """
            If no hyperbox present initially so create new
        """
        if len(self.V)==0 and len(self.W)==0 :
            self.V.append(xh_l)
            self.W.append(xh_u)
            self.hyperbox_class.append(d)
            self.U.append(tmp)
            expand = False
            return len(self.V)-1 , expand 
                
        """
            returns the most sutaible hyperbox for input pattern x
            otherwise None
        """
        mylist = []
        
        for i in range(len(self.V)):
            if self.hyperbox_class[i]==[0]:
                test = [0 for _ in range(len(xh_l))]
                for _ in range(len(xh_l)):
                    if self.theta >= (max(self.W[i][_],xh_u[_])-min(self.V[i][_],xh_l[_])):
                        test[_]=1
                    else:
                        pass
                if test == [1 for _ in range(len(x))] :
                    expand = True
                    """
                        Label hyperbox with class label of input pattern
                    """
                    self.hyperbox_class[i][0]=d[0] 
                    return i,expand
            elif self.hyperbox_class[i]==d:
                mylist.append((self.fuzzy_membership(x,self.V[i],self.W[i])))
            else:
                mylist.append(-1)
                
        if len(mylist)>0:
            for box in sorted(mylist)[::-1]:
                i = mylist.index(box)
                test = [0 for i in range(len(xh_l))]
                for _ in range(len(xh_l)):
                    if self.theta >= (max(self.W[i][_],xh_u[_])-min(self.V[i][_],xh_l[_])):
                        test[_]=1
                    else:
                        pass

                if test == [1 for _ in range(len(x))] :
                    expand = True
                    return i,expand

            '''
                No hyperbox follow expansion criteria so create new
            '''
            self.V.append(xh_l)
            self.W.append(xh_u)
            self.hyperbox_class.append(d)
            self.U.append(tmp)
            expand = False
            return len(self.V)-1 , expand
            
        else:
            """
                If no hyperbox present for pattern x of class d so create new 
            """
            self.V.append(xh_l)
            self.W.append(xh_u)
            self.hyperbox_class.append(d)
            self.U.append(tmp)
            expand = False
            return len(self.V)-1,expand
    
    def expand(self,x,key):
        xh_l,xh_u = x[0],x[1]
        self.V[key] = [min(self.V[key][i],xh_l[i]) for i in range(len(xh_l))]
        self.W[key] = [max(self.W[key][i],xh_u[i]) for i in range(len(xh_u))]
        
    
    def overlap_Test(self):
        del_old = 1
        del_new = 1
        box_1,box_2,delta = -1,-1,-1
        for j in range(len(self.V)):
            if self.hyperbox_class[j] == [0] :
                for k in range(j+1,len(self.V)):
                    for i in range(len(self.V[j])):
                    
                        """
                            Test Four cases given by Patrick Simpson
                        """
                    
                        if (self.V[j][i] < self.V[k][i] < self.W[j][i] < self.W[k][i]) :
                               del_new = min(del_old,self.V[j][i]-self.V[k][i])
                        elif (self.V[k][i] < self.V[j][i] < self.W[k][i] < self.W[j][i]) :
                               del_new = min(del_old,self.W[k][i]-self.V[j][i])
                        elif (self.V[j][i] < self.V[k][i] < self.W[k][i] < self.W[j][i]) :
                               del_new = min(del_old,min(self.W[k][i]-self.V[j][i],                                                     self.W[j][i]-self.V[k][i]))
                        elif (self.V[k][i] < self.V[j][i] < self.W[j][i] < self.W[k][i]) :
                               del_new = min(del_old,min(self.W[j][i]-self.V[k][i],                                                     self.W[k][i]-self.V[j][i]))
                       
                        """
                            Check dimension for which overlap is minimum
                        """
                        #print(del_old , del_new , del_old-del_new , i)
                        if del_old - del_new > 0.0 :
                            delta = i
                            box_1,box_2 = j,k
                            del_old = del_new
                            del_new = 1
                        else:
                            pass
                        
            else:
                for k in range(j+1,len(self.V)):
                    if self.hyperbox_class[j]==self.hyperbox_class[k] :
                        pass
                    else:
                        for i in range(len(self.V[j])):

                            """
                                Test Four cases given by Patrick Simpson
                            """

                            if (self.V[j][i] < self.V[k][i] < self.W[j][i] < self.W[k][i]) :
                                   del_new = min(del_old,self.V[j][i]-self.V[k][i])
                            elif (self.V[k][i] < self.V[j][i] < self.W[k][i] < self.W[j][i]) :
                                   del_new = min(del_old,self.W[k][i]-self.V[j][i])
                            elif (self.V[j][i] < self.V[k][i] < self.W[k][i] < self.W[j][i]) :
                                   del_new = min(del_old,min(self.W[k][i]-self.V[j][i],                                                         self.W[j][i]-self.V[k][i]))
                            elif (self.V[k][i] < self.V[j][i] < self.W[j][i] < self.W[k][i]) :
                                   del_new = min(del_old,min(self.W[j][i]-self.V[k][i],                                                         self.W[k][i]-self.V[j][i]))

                            """
                                Check dimension for which overlap is minimum
                            """
                            #print(del_old , del_new , del_old-del_new , i)
                            if del_old - del_new > 0.0 :
                                delta = i
                                box_1,box_2 = j,k
                                del_old = del_new
                                del_new = 1
                            else:
                                pass
                
        return delta , box_1, box_2
            
                       
    def contraction(self,delta,box_1,box_2):
        if (self.V[box_1][delta] < self.V[box_2][delta] < self.W[box_1][delta] <             self.W[box_2][delta]) :
            self.W[box_1][delta] = (self.W[box_1][delta]+self.V[box_2][delta])/2
            self.V[box_2][delta] = (self.W[box_1][delta]+self.V[box_2][delta])/2 
            
        elif (self.V[box_2][delta] < self.V[box_1][delta] < self.W[box_2][delta] <               self.W[box_1][delta]) :
            self.W[box_2][delta] = (self.W[box_2][delta]+self.V[box_1][delta])/2
            self.V[box_1][delta] = (self.W[box_2][delta]+self.V[box_1][delta])/2
            
        elif (self.V[box_1][delta] < self.V[box_2][delta] < self.W[box_2][delta] <               self.W[box_1][delta]) :
            if (self.W[box_2][delta]-self.V[box_1][delta])<             (self.W[box_1][delta]-self.V[box_2][delta]):
                self.V[box_1][delta] = self.W[box_2][delta]
            else:
                self.W[box_1][delta] = self.V[box_2][delta]
            
        elif (self.V[box_2][delta] < self.V[box_1][delta] < self.W[box_1][delta] <               self.W[box_2][delta]) :
            if (self.W[box_2][delta]-self.V[box_1][delta])<            (self.W[box_1][delta]-self.V[box_2][delta]):
                self.W[box_2][delta] = self.V[box_1][delta]
            else:
                self.V[box_2][delta] = self.W[box_1][delta]
    
    def predict(self,x):
        mylist = []
        for i in range(len(self.V)):
            mylist.append([self.fuzzy_membership(x,self.V[i],self.W[i])])
            
        result = np.multiply(mylist,self.U)
        prediction = np.zeros([self.clasess])
        for i in range(self.clasess):
            #print('pattern {} belongs to class {} with fuzzy membership value : {}'.format(x,i+1,max(result[:,i])))
            prediction[i] = max(result[:, i])
        
        return prediction.argmax(axis=0)
                    
    def train(self,X,d_,epochs):
        self.clasess = len(np.unique(np.array(d_)))
        for _ in range(epochs):
            print('epoch : {}'.format(_+1))
            print('='*70)

            for x,d in zip(X,d_):
                '''Get most sutaible hyperbox!!'''
                i , expand = self.get_hyperbox(x,d)

                #print('input pattern : ',x , d)
                #print('Hyperbox : {} , {} '.format(self.V[i],self.W[i])  )

                if expand:
                    self.expand(x,i)
                    #print("Expanded Hyperbox : ",self.V[i] , self.W[i])
                    
                    """
                     As hyperbox expanded cheak overlap and contract according 
                     to cases given by Gabrys and Bargiela
                    """
                    self.overlap_Test()
                    

                #print('='*70)
            
        #print('final hyperbox : ')
        #print('V : ',self.V)
        #print('W : ',self.W)

In [7]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('../dataset/training_data_LSTM_FS_3_belt.csv')
normalized_df = (df - df.min()) / (df.max() - df.min())
normalized_df['1'] = df['1']
data = normalized_df.as_matrix()

train, test = train_test_split(data, test_size=0.2)
print(train.shape)
print(test.shape)

n_input = 42     # 입력데이터의 속성 수
n_step = 5       # 한 번에 입력할 데이터 수
n_class = 11     # 결과값의 종류 수 (Activity 1~11)
batch_size = 1   # 배치 크기 (ex. n_step개 데이터 batch_size묶음이 1배치)
n_epoch = 50    # 총 학습 횟수

# epoch 당 batch 수 계산
train_batch = train.shape[0]//(batch_size*n_step)
print(train_batch)
test_batch = test.shape[0]//(batch_size*n_step)
print(test_batch)

x_train_temp = train[:, :n_input]
y_train_temp = train[:, -1]

x_train = np.zeros([train_batch, n_step, n_input])
y_train = np.zeros([train_batch, 1], dtype=int)
for i in range(train_batch):
    for j in range(n_step):
        for k in range(n_input):
            x_train[i][j][k] = x_train_temp[i + j][k]
    y_train[i][0] = int(y_train_temp[n_step * i])

x_test_temp = test[:, :n_input]
y_test_temp = test[:, -1]

x_test = np.zeros([test_batch, n_step, n_input])
y_test = np.zeros([test_batch, 1], dtype=int)
for i in range(test_batch):
    for j in range(n_step):
        for k in range(n_input):
            x_test[i][j][k] = x_test_temp[i + j][k]
    y_test[i][0] = y_test_temp[n_step * i]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

x_train = x_train.tolist()
y_train = y_train.tolist()
x_test = x_test.tolist()
y_test = y_test.tolist()

(1607, 44)
(402, 44)
321
80
(321, 5, 42)
(321, 1)
(80, 5, 42)
(80, 1)


In [8]:
fuzzy = GFuzzyMinMaxNN(1, theta=0.4)
fuzzy.train(x_train, y_train, 50)

epoch : 1
epoch : 2
epoch : 3
epoch : 4
epoch : 5
epoch : 6
epoch : 7
epoch : 8
epoch : 9
epoch : 10
epoch : 11
epoch : 12
epoch : 13
epoch : 14
epoch : 15
epoch : 16
epoch : 17
epoch : 18
epoch : 19
epoch : 20
epoch : 21
epoch : 22
epoch : 23
epoch : 24
epoch : 25
epoch : 26
epoch : 27
epoch : 28
epoch : 29
epoch : 30
epoch : 31
epoch : 32
epoch : 33
epoch : 34
epoch : 35
epoch : 36
epoch : 37
epoch : 38
epoch : 39
epoch : 40
epoch : 41
epoch : 42
epoch : 43
epoch : 44
epoch : 45
epoch : 46
epoch : 47
epoch : 48
epoch : 49
epoch : 50


In [9]:
accuracy = 0.0
for i in range(test_batch):
    if(fuzzy.predict(x_test[i]) == y_test[i]):
        accuracy += 1
accuracy /= test_batch

print(accuracy)

0.0875


In [2]:
class FuzzyMMC:

	def __init__(self, sensitivity=1, exp_bound=1, animate=False):
		'''
		Constructor for FuzzyMMC class
		'''
		self.sensitivity = sensitivity
		self.hyperboxes = None
		self.isanimate = animate
		self.classes = np.array([])
		self.exp_bound = exp_bound

		if self.animate:
			self.box_history = []
			self.train_patterns = []


	def membership(self, pattern):
		'''
		Calculates membership values a pattern
		
		Returns an ndarray of membership values of all hyperboxes
		'''
		min_pts = self.hyperboxes[:, 0, :]
		max_pts = self.hyperboxes[:, 1, :]
		
		a = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, pattern - max_pts)))))
		b = np.maximum(0, (1 - np.maximum(0, (self.sensitivity * np.minimum(1, min_pts - pattern)))))

		return np.sum(a + b, axis=1) / (2 * len(pattern))


	def overlap_contract(self, index):
		'''
		Check if any classwise dissimilar hyperboxes overlap
		'''
		contracted = False
		for test_box in range(len(self.hyperboxes)):

			if self.classes[test_box] == self.classes[index]:
				# Ignore same class hyperbox overlap
				continue
	
			expanded_box = self.hyperboxes[index]
			box = self.hyperboxes[test_box]

			## TODO: Refactor for vectorization
			vj, wj = expanded_box
			vk, wk = box

			delta_new = delta_old = 1
			min_overlap_index = -1
			for i in range(len(vj)):
				if vj[i] < vk[i] < wj[i] < wk[i]:
					delta_new = min(delta_old, wj[i] - vk[i])

				elif vk[i] < vj[i] < wk[i] < wj[i]:
					delta_new = min(delta_old, wk[i] - vj[i])

				elif vj[i] < vk[i] < wk[i] < wj[i]:
					delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

				elif vk[i] < vj[i] < wj[i] < wk[i]:
					delta_new = min(delta_old, min(wj[i] - vk[i], wk[i] - vj[i]))

				if delta_old - delta_new > 0:
					min_overlap_index = i
					delta_old = delta_new

			if min_overlap_index >= 0:
				i = min_overlap_index
				# We need to contract the expanded box
				if vj[i] < vk[i] < wj[i] < wk[i]:
					vk[i] = wj[i] = (vk[i] + wj[i])/2

				elif vk[i] < vj[i] < wk[i] < wj[i]:
					vj[i] = wk[i] = (vj[i] + wk[i])/2

				elif vj[i] < vk[i] < wk[i] < wj[i]:
					if (wj[i] - vk[i]) > (wk[i] - vj[i]):
						vj[i] = wk[i]

					else:
						wj[i] = vk[i]

				elif vk[i] < vj[i] < wj[i] < wk[i]:
					if (wk[i] - vj[i]) > (wj[i] - vk[i]):
						vk[i] = wj[i]

					else:
						wk[i] = vj[i]

				self.hyperboxes[test_box] = np.array([vk, wk])
				self.hyperboxes[index] = np.array([vj, wj])
				contracted = True

		return contracted



	def train_pattern(self, X, Y):
		'''
		Main function that trains a fuzzy min max classifier
		Note:
		Y is a one-hot encoded target variable
		'''
		target = Y
		
		if target not in self.classes:
			
			# Create a new hyberbox
			if self.hyperboxes is not None:
				self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
				self.classes = np.hstack((self.classes, np.array([target])))

			else:
				self.hyperboxes = np.array([[X, X]])
				self.classes = np.array([target])

			if self.isanimate:
				self.box_history.append(np.copy(self.hyperboxes))
				self.train_patterns.append((X, Y))
		else:
			
			memberships = self.membership(X)
			memberships[np.where(self.classes != target)] = 0
			memberships = sorted(list(enumerate(memberships)), key=lambda x: x[1], reverse=True)
			
			# Expand the most suitable hyperbox
			count = 0
			while True:
				index = memberships[count][0]
				min_new = np.minimum(self.hyperboxes[index, 0, :], X)
				max_new = np.maximum(self.hyperboxes[index, 1, :], X)
				
				if self.exp_bound * len(np.unique(self.classes)) >= np.sum(max_new - min_new):
					self.hyperboxes[index, 0] = min_new
					self.hyperboxes[index, 1] = max_new
					break
				else:
					count += 1

				if count == len(memberships):
					self.hyperboxes = np.vstack((self.hyperboxes, np.array([[X, X]])))
					self.classes = np.hstack((self.classes, np.array([target])))
					index = len(self.hyperboxes) - 1
					break

			# Overlap test
			if self.isanimate:
				self.box_history.append(np.copy(self.hyperboxes))
				self.train_patterns.append((X, Y))

			contracted = self.overlap_contract(index)
			
			if self.isanimate and contracted:	
				self.box_history.append(np.copy(self.hyperboxes))
				self.train_patterns.append((X, Y))


	def fit(self, X, Y):
		'''
		Wrapper for train_pattern
		'''
		for x, y in zip(X, Y):
			self.train_pattern(x, y)


	def predict(self, X):
		'''
		Predict the class of the pattern X
		'''
		classes = np.unique(self.classes)
		results = []
		memberships = self.membership(X)
		max_prediction = 0
		pred_class = 0
		for _class in classes:
			mask = np.zeros((len(self.hyperboxes),))
			mask[np.where(self.classes == _class)] = 1
			p = memberships * mask
			prediction, class_index = np.max(p), np.argmax(p)
			if prediction > max_prediction:
				max_prediction = prediction
				pred_class = class_index

		return max_prediction, self.classes[pred_class] 


	def score(self, X, Y):
		'''
		Scores the classifier
		'''
		count = 0
		for x, y in zip(X, Y):
			_, pred = self.predict(x)
			if y == pred:
				count += 1

		return count / len(Y)


	def animate(self, frame_rate=10, filename='', verbose=True):
		'''
		To make a video of the classifier training.
		NOTE: Only possible when working with 2 dimensional patterns
		'''
		if self.isanimate:
			animator = Animator(box_history=self.box_history,
								train_patterns=self.train_patterns,
								classes=self.classes,
								frame_rate=frame_rate,
								exp_bound=self.exp_bound,
								sensitivity=self.sensitivity,
								filename=filename,
								verbose=verbose)

			animator.animate()

			return animator.filename

		else:
			raise Exception('No animation data was collected! Create a fuzzy classifier instance with animate=True')

In [4]:
from sklearn.model_selection import train_test_split

df = pd.read_csv('../dataset/training_data_LSTM_FS_3_belt.csv')
normalized_df = (df - df.min()) / (df.max() - df.min())
normalized_df['1'] = df['1']
data = normalized_df.as_matrix()

train, test = train_test_split(data, test_size=0.2)
print(train.shape)
print(test.shape)

n_input = 43     # 입력데이터의 속성 수
n_step = 5       # 한 번에 입력할 데이터 수
n_class = 11     # 결과값의 종류 수 (Activity 1~11)
batch_size = 1   # 배치 크기 (ex. n_step개 데이터 batch_size묶음이 1배치)
n_epoch = 50    # 총 학습 횟수

# epoch 당 batch 수 계산
train_batch = train.shape[0]//(batch_size*n_step)
print(train_batch)
test_batch = test.shape[0]//(batch_size*n_step)
print(test_batch)

x_train_temp = train[:, :n_input]
y_train_temp = train[:, -1]

x_train = np.zeros([train_batch, n_step * n_input])
y_train = np.zeros([train_batch], dtype=int)
for i in range(train_batch):
    for j in range(n_step):
        for k in range(n_input):
            x_train[i][n_input * j + k] = x_train_temp[i + j][k]
    y_train[i] = int(y_train_temp[n_step * i])

x_test_temp = test[:, :n_input]
y_test_temp = test[:, -1]

x_test = np.zeros([test_batch, n_step * n_input])
y_test = np.zeros([test_batch], dtype=int)
for i in range(test_batch):
    for j in range(n_step):
        for k in range(n_input):
            x_test[i][n_input * j + k] = x_test_temp[i + j][k]
    y_test[i] = y_test_temp[n_step * i]

print(x_train.shape)
print(y_train.shape)
print(x_test.shape)
print(y_test.shape)

(1607, 44)
(402, 44)
321
80
(321, 215)
(321,)
(80, 215)
(80,)


In [5]:
model = FuzzyMMC(sensitivity=1, exp_bound = 0.7)
for i in range(1):
    print("Epoch: ", i + 1)
    model.fit(x_train, y_train)
print("Train Accuracy = ", model.score(x_train, y_train))
print("Test Accuracy = ", model.score(x_test, y_test))

Epoch:  1
Train Accuracy =  1.0
Test Accuracy =  0.1
